In [6]:
import pandas as pd
import numpy as np
import re
import warnings
warnings.filterwarnings('ignore')

In [7]:
#path to 'gene_presence_absence.Rtab' file produced by Roary; https://sanger-pathogens.github.io/Roary/
rast_rtab = pd.read_csv('/home/jee/bvbrc_workshop_Aug2022/bvbrc_annot/rast_gff/roary_output/gene_presence_absence.Rtab', sep='\t')

###  Pangenome

In [12]:
rast_df = rast_rtab.rename(columns=lambda x: re.sub('.rast','',x)).set_index('Gene').T
X = rast_df.reset_index().rename(columns={'index':'Run_accession'})
X

Gene,Run_accession,"LSU ribosomal protein L33p @ LSU ribosomal protein L33p, zinc-dependent",SSU ribosomal protein S16p,Translation initiation factor 3,SSU ribosomal protein S9p (S16e),Cold shock protein of CSP family,SSU ribosomal protein S21p,LSU ribosomal protein L13p (L13Ae),SSU ribosomal protein S5p (S2e),SSU ribosomal protein S8p (S15Ae),...,group_9967,group_9969,group_9970,group_9975,group_9976,group_9981,group_9982,group_9984,group_9996,group_9997
0,SRR13712361,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1,SRR13712362,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
2,SRR13712363,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
3,SRR13712364,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
4,SRR13712365,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
642,SRR14026551,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
643,SRR14026552,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
644,SRR14026553,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
645,SRR14026554,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0


In [17]:
donor_recipient_ls = ['SRR14026472','SRR14026519']
# donor_recipient = X[X['Run_accession'].isin(donor_recipient_ls)].set_index('Run_accession').T

In [24]:
temp = donor_recipient[(donor_recipient['SRR14026472']==0) & (donor_recipient['SRR14026519']==1)].reset_index()
temp[~temp['Gene'].str.contains('group', case=False)].to_csv('only_in_NS1659_SRR14026519.csv', index=None)

#### AST metadata

In [13]:
meta_all = pd.read_csv('/home/jee/enterococcus_sample_reconciliation/all_combined_enterococcus_metadata.tsv',
                  sep='\t').rename({'Quinolone':'Quin'},axis=1)

meta = pd.read_csv('/home/jee/enterococcus_sample_reconciliation/all_combined_enterococcus_metadata.tsv',
                  sep='\t').rename({'Quinolone':'Quin'},axis=1)[['Run_accession', 'Species',
                  'Ampicillin', 'Vancomycin', 'Teicoplanin', 'Doxycycline',
                  'Erythromycin', 'Nitrofurantoin', 'Gentamicin', 'Linezolid',
                  'Levofloxacin', 'Quin', 'Streptomycin',
                  'Tigecycline']]

In [14]:
Xy_van = X.merge(meta[['Run_accession','Vancomycin']], on = 'Run_accession')
Xy_doxy =  X.merge(meta[['Run_accession','Doxycycline']], on = 'Run_accession')
Xy_eryth =  X.merge(meta[['Run_accession','Erythromycin']], on = 'Run_accession')

In [15]:
X_Vancomycin =Xy_van[Xy_van['Vancomycin'].isin(['R','S','S '])].replace(['S'], 0).replace(['S '], 0).replace(['R'],1).set_index('Run_accession')
Xy_Doxycycline = Xy_doxy[Xy_doxy['Doxycycline'].isin(['R','S'])].replace(['S'], 0).replace(['R'],1).set_index('Run_accession')
Xy_Erythromycin = Xy_eryth[Xy_eryth['Erythromycin'].isin(['R','S'])].replace(['S'], 0).replace(['R'],1).set_index('Run_accession')

In [16]:
# intermediate isolates for model combining both species (E. faecium & E. faecalis)
intermediate_Vancomycin = Xy_van[Xy_van['Vancomycin'].isin(['I'])].drop('Vancomycin', axis=1).set_index('Run_accession')
intermediate_Doxycycline = Xy_doxy[Xy_doxy['Doxycycline'].isin(['I'])].drop('Doxycycline', axis=1).set_index('Run_accession')
intermediate_Erythromycin = Xy_eryth[Xy_eryth['Erythromycin'].isin(['I'])].drop('Erythromycin', axis=1).set_index('Run_accession')

---

#### ML functions

In [20]:
from sklearn.feature_selection import SelectKBest, chi2, mutual_info_classif

from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold

from sklearn.metrics import classification_report,confusion_matrix,roc_curve,auc,precision_recall_curve,roc_curve

import itertools as it

In [22]:
from sklearn import metrics
from sklearn.metrics import classification_report

gb_clf = GradientBoostingClassifier(n_estimators = 100, max_features = 'auto', criterion = 'friedman_mse', 
                                     loss = 'exponential', learning_rate = 0.1)

rf_clf = RandomForestClassifier(n_estimators = 500, max_features = 'auto', criterion='gini')

def ML_features(X,y,clf):
    
    ###############################################
    #Get feature names first:
    features=[]
    for columns in X.columns:
        features.append(columns)
    ###############################################
    
    #init a kfold object, "skf"
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

    prediction_outcome_ls = []
    
    #So that we can plot again, we'll make a nice data structure for our results
    records = []
    confusion_matrices = []
    
    feat_ls = []
    
    #looping on the skf.split() provides train/test indices
    for train_index, test_index in skf.split(X.values,y.values):
        clf.fit(X.values[train_index], y.values[train_index])
        
        ###############################################
        #figure out the feature importance 
        model = clf.fit(X.values[train_index], y.values[train_index])
        imp_features = model.feature_importances_
        df_imp_features = pd.DataFrame({"features":features}).join(pd.DataFrame({"weights":imp_features}))
        df_imp_features_final = df_imp_features.sort_values(by=['weights'], ascending=False)
        display(df_imp_features_final)
        feat_ls.append(df_imp_features_final)
        ###############################################
        
                
        y_true_index = y.index[test_index]
        y_true = y.values[test_index]
        y_pred = clf.predict(X.values[test_index]) #get the predicted labels of the test set.
        
        y_true_index_ls = pd.DataFrame(y_true_index.to_list())
        y_true_df = pd.DataFrame(y_true)
        y_pred_df = pd.DataFrame(y_pred)

        #concatenating the two dataframes
        y_lists = [y_true_index_ls, y_true_df, y_pred_df]
        df_y_outcomes = pd.concat(y_lists, axis=1)
        df_y_outcomes.columns = ['GenomeID','y_true','y_pred']
        df_y_outcomes

        prediction_outcome_ls.append(df_y_outcomes)

  
   
        #Calculate metrics individually  
        accuracy = metrics.accuracy_score(y_true, y_pred)
        f1 = metrics.f1_score(y_true, y_pred, average='weighted') #F1 behaves different for binary classification problems! be careful
        precision = metrics.precision_score(y_true, y_pred, average='weighted')
        recall = metrics.recall_score(y_true, y_pred, average='weighted')

        cm = metrics.confusion_matrix(y_true, y_pred) # pass your labels here if you want them.

        #Keeping the scores as a list of dicts that all have the same keys 
        records.append({'Accuracy': accuracy,
                        'F1': f1,
                        'Precision': precision,
                        'Recall': recall})

        confusion_matrices.append(cm)

    #Data summary

    #metrics is the same as before
    df = pd.DataFrame.from_records(records)
    #sum our confusion matrix 
    cm = np.asarray(confusion_matrices).sum(axis=0)

    #Misclassified cases
    prediction_outcome = pd.concat([prediction_outcome_ls[0], prediction_outcome_ls[1], prediction_outcome_ls[2],
                                   prediction_outcome_ls[3], prediction_outcome_ls[4]], axis=0).set_index('GenomeID')
    misclassified = prediction_outcome[prediction_outcome.apply(lambda x: x['y_true'] != x['y_pred'], axis = 1)].index
    
    print('{antibiotic} resistance prediction,'.format(antibiotic=y.columns[0]))
    print(df)
    print()
    print("The misclassified isolates for {ab} resistance prediction is stated in a below list:".format(ab=y.columns[0]))
    print(misclassified)   
    
    return df, feat_ls, model, misclassified
           #[0] = the dataframe , [1] = feature list, [2] = clf.ft... (the trained model!)

In [23]:
def feat_organization(trained_model):
    fs_list = []

    for i in range(5):
        temp = trained_model[1][i].iloc[0:5]
        fs_list.append(temp)

    dfs = pd.concat(fs_list).groupby('features')['weights'].agg(list).reset_index()
    dfs = dfs.sort_values(by=['weights'], ascending=False)
    display(dfs)
    
    return dfs 

---

### ML results

In [25]:
X_Vancomycin.iloc[:,0:34819] #with the vancomycin resistance/susceptible metadata column 

,"LSU ribosomal protein L33p @ LSU ribosomal protein L33p, zinc-dependent",SSU ribosomal protein S16p,Translation initiation factor 3,SSU ribosomal protein S9p (S16e),Cold shock protein of CSP family,SSU ribosomal protein S21p,LSU ribosomal protein L13p (L13Ae),SSU ribosomal protein S5p (S2e),SSU ribosomal protein S8p (S15Ae),"SSU ribosomal protein S14p (S29e) @ SSU ribosomal protein S14p (S29e), zinc-dependent",...,group_9969,group_9970,group_9975,group_9976,group_9981,group_9982,group_9984,group_9996,group_9997,Vancomycin
Run_accession,,,,,,,,,,,,,,,,,,,,,
SRR13712361,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
SRR13712362,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
SRR13712363,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
SRR13712364,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
SRR13712365,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SRR14026551,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
SRR14026552,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
SRR14026553,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0


In [17]:
# March 8th, 2023

van_rast_ML_gb = ML_features(X_Vancomycin.iloc[:,0:34818],X_Vancomycin[['Vancomycin']], gb_clf)

/home/jee/conda/envs/ml_prediction/lib/python3.7/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/home/jee/conda/envs/ml_prediction/lib/python3.7/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


,features,weights
5534,"D-lactate dehydrogenase VanH, associated with ...",0.399422
5532,D-alanyl-D-alanine dipeptidase (EC 3.4.13.22) ...,0.271038
5533,D-alanine--(R)-lactate ligase (EC 6.1.2.1) > VanA,0.257924
16483,group_3602,0.018153
29138,group_27863,0.003930
...,...,...
11649,FIG00631911: hypothetical protein,0.000000
11648,group_17940,0.000000
11647,group_17840,0.000000
11646,group_17821,0.000000


/home/jee/conda/envs/ml_prediction/lib/python3.7/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/home/jee/conda/envs/ml_prediction/lib/python3.7/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


,features,weights
5534,"D-lactate dehydrogenase VanH, associated with ...",0.514364
5533,D-alanine--(R)-lactate ligase (EC 6.1.2.1) > VanA,0.233582
5532,D-alanyl-D-alanine dipeptidase (EC 3.4.13.22) ...,0.193673
28749,group_27459,0.014778
3280,group_8940,0.004863
...,...,...
11630,group_16604,0.000000
11629,group_16576,0.000000
11628,group_16495,0.000000
11627,group_16486,0.000000


/home/jee/conda/envs/ml_prediction/lib/python3.7/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/home/jee/conda/envs/ml_prediction/lib/python3.7/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


...

/home/jee/conda/envs/ml_prediction/lib/python3.7/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/home/jee/conda/envs/ml_prediction/lib/python3.7/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


,features,weights
5533,D-alanine--(R)-lactate ligase (EC 6.1.2.1) > VanA,4.496214e-01
5532,D-alanyl-D-alanine dipeptidase (EC 3.4.13.22) ...,3.986886e-01
5534,"D-lactate dehydrogenase VanH, associated with ...",1.084402e-01
29181,group_27905,4.106828e-03
3903,group_9855,3.806954e-03
...,...,...
4226,group_11062,-5.010570e-19
36,Mobile element protein,-3.598957e-18
35,"PTS system, gluconate-specific IIB component",-7.138532e-18
3635,group_3921,-8.053071e-18


/home/jee/conda/envs/ml_prediction/lib/python3.7/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/home/jee/conda/envs/ml_prediction/lib/python3.7/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


,features,weights
5533,D-alanine--(R)-lactate ligase (EC 6.1.2.1) > VanA,4.756292e-01
5534,"D-lactate dehydrogenase VanH, associated with ...",3.572093e-01
5532,D-alanyl-D-alanine dipeptidase (EC 3.4.13.22) ...,1.101120e-01
4115,group_5202,1.816221e-02
28749,group_27459,1.462787e-02
...,...,...
3649,group_12943,-2.479457e-19
5315,group_10601,-1.275887e-18
36,Mobile element protein,-2.418271e-18
4662,group_5123,-3.564220e-18


Vancomycin resistance prediction,
   Accuracy        F1  Precision    Recall
0  0.967213  0.967831   0.969285  0.967213
1  0.991803  0.991878   0.992176  0.991803
2  0.991803  0.991878   0.992176  0.991803
3  0.983471  0.983471   0.983471  0.983471
4  0.983471  0.983117   0.983792  0.983471

The misclassified isolates for Vancomycin resistance prediction is stated in a below list:
Index(['SRR13727033', 'SRR13999997', 'SRR14000023', 'SRR14011043',
       'SRR14010988', 'SRR14026552', 'SRR14000586', 'SRR14011017',
       'SRR14000612', 'SRR14011035'],
      dtype='object', name='GenomeID')


In [52]:
van_rf_rast_model = ML_features(X_Vancomycin.iloc[:,0:34818],X_Vancomycin[['Vancomycin']], rf_clf)

/home/jee/conda/envs/ml_prediction/lib/python3.7/site-packages/ipykernel_launcher.py:32: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
/home/jee/conda/envs/ml_prediction/lib/python3.7/site-packages/ipykernel_launcher.py:36: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


,features,weights
5533,D-alanine--(R)-lactate ligase (EC 6.1.2.1) > VanA,0.009461
5618,group_8577,0.009402
5375,Putrescine transport ATP-binding protein PotA ...,0.009098
5369,group_18896,0.007707
5564,group_2780,0.007361
...,...,...
13339,group_5011,0.000000
13338,group_4994,0.000000
13336,group_4705,0.000000
13335,group_4661,0.000000


/home/jee/conda/envs/ml_prediction/lib/python3.7/site-packages/ipykernel_launcher.py:32: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
/home/jee/conda/envs/ml_prediction/lib/python3.7/site-packages/ipykernel_launcher.py:36: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


,features,weights
5374,FIG00628452: hypothetical protein,0.011121
5042,group_914,0.010865
5532,D-alanyl-D-alanine dipeptidase (EC 3.4.13.22) ...,0.009494
5534,"D-lactate dehydrogenase VanH, associated with ...",0.009386
5212,group_18932,0.008504
...,...,...
13404,group_9255,0.000000
13403,group_9170,0.000000
13402,group_9126,0.000000
13401,group_907,0.000000


/home/jee/conda/envs/ml_prediction/lib/python3.7/site-packages/ipykernel_launcher.py:32: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
/home/jee/conda/envs/ml_prediction/lib/python3.7/site-packages/ipykernel_launcher.py:36: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


...

/home/jee/conda/envs/ml_prediction/lib/python3.7/site-packages/ipykernel_launcher.py:32: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
/home/jee/conda/envs/ml_prediction/lib/python3.7/site-packages/ipykernel_launcher.py:36: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


,features,weights
5533,D-alanine--(R)-lactate ligase (EC 6.1.2.1) > VanA,0.012063
5375,Putrescine transport ATP-binding protein PotA ...,0.011175
5369,group_18896,0.009286
5627,Vancomycin (or other glycopeptides) response r...,0.008190
4811,group_3268,0.008102
...,...,...
13090,group_16215,0.000000
13088,group_16103,0.000000
13087,group_16084,0.000000
13085,group_1595,0.000000


/home/jee/conda/envs/ml_prediction/lib/python3.7/site-packages/ipykernel_launcher.py:32: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
/home/jee/conda/envs/ml_prediction/lib/python3.7/site-packages/ipykernel_launcher.py:36: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


,features,weights
5627,Vancomycin (or other glycopeptides) response r...,0.012940
5375,Putrescine transport ATP-binding protein PotA ...,0.010665
5532,D-alanyl-D-alanine dipeptidase (EC 3.4.13.22) ...,0.010408
6239,group_28650,0.009586
6250,Vancomycin (or other glycopeptides) histidine ...,0.009494
...,...,...
13295,group_30904,0.000000
13294,group_30903,0.000000
13293,group_30902,0.000000
13292,group_30901,0.000000


Vancomycin resistance prediction,
   Accuracy        F1  Precision    Recall
0  0.991803  0.991883   0.992194  0.991803
1  0.991803  0.991878   0.992176  0.991803
2  0.975410  0.975635   0.976073  0.975410
3  0.975207  0.975447   0.975911  0.975207
4  0.966942  0.967564   0.969029  0.966942

The misclassified isolates for Vancomycin resistance prediction is stated in a below list:
Index(['SRR14026550', 'SRR14010988', 'SRR14010973', 'SRR14026521',
       'SRR14026552', 'SRR14000586', 'SRR14011017', 'SRR14026517',
       'SRR14000612', 'SRR14026519', 'SRR14026545', 'SRR14026554'],
      dtype='object', name='GenomeID')


In [17]:
# March 8th, 2023
van_gb_ML_df = van_rast_ML_gb[0]
van_gb_ML_df.loc['Mean score'] = van_gb_ML_df.mean()
van_gb_ML_df.loc['std'] = van_gb_ML_df.std()
van_gb_ML_df['Classifier'] = 'Gradient boosting'
van_gb_ML_df['Antibiotic'] = 'Vancomycin'
van_gb_ML_df['Features'] = 'Rast Pangenome'

van_rf_ML_df = van_rast_ML_rf[0]
van_rf_ML_df.loc['Mean score'] = van_rf_ML_df.mean()
van_rf_ML_df.loc['std'] = van_rf_ML_df.std()
van_rf_ML_df['Classifier'] = 'Random forest'
van_rf_ML_df['Antibiotic'] = 'Vancomycin'
van_rf_ML_df['Features'] = 'Rast Pangenome'

**Results; Vancomycin - gradient boosting**

\begin{tabular}{lrrrrlll}
\toprule
{} &  Accuracy &        F1 &  Precision &    Recall &         Classifier &  Antibiotic &        Features \\
\midrule
0          &  0.967213 &  0.967831 &   0.969285 &  0.967213 &  Gradient boosting &  Vancomycin &  Rast Pangenome \\
1          &  0.991803 &  0.991878 &   0.992176 &  0.991803 &  Gradient boosting &  Vancomycin &  Rast Pangenome \\
2          &  0.991803 &  0.991878 &   0.992176 &  0.991803 &  Gradient boosting &  Vancomycin &  Rast Pangenome \\
3          &  0.983471 &  0.983471 &   0.983471 &  0.983471 &  Gradient boosting &  Vancomycin &  Rast Pangenome \\
4          &  0.983471 &  0.983117 &   0.983792 &  0.983471 &  Gradient boosting &  Vancomycin &  Rast Pangenome \\
Mean score &  0.983552 &  0.983635 &   0.984180 &  0.983552 &  Gradient boosting &  Vancomycin &  Rast Pangenome \\
std        &  0.008979 &  0.008786 &   0.008371 &  0.008979 &  Gradient boosting &  Vancomycin &  Rast Pangenome \\
\bottomrule
\end{tabular}

\begin{tabular}{lrrrrlll}
\toprule
{} &  Accuracy &        F1 &  Precision &    Recall &     Classifier &  Antibiotic &        Features \\
\midrule
0          &  0.983607 &  0.983607 &   0.983607 &  0.983607 &  Random forest &  Vancomycin &  Rast Pangenome \\
1          &  0.983607 &  0.983897 &   0.985032 &  0.983607 &  Random forest &  Vancomycin &  Rast Pangenome \\
2          &  0.975410 &  0.975635 &   0.976073 &  0.975410 &  Random forest &  Vancomycin &  Rast Pangenome \\
3          &  0.975207 &  0.975447 &   0.975911 &  0.975207 &  Random forest &  Vancomycin &  Rast Pangenome \\
4          &  0.966942 &  0.967564 &   0.969029 &  0.966942 &  Random forest &  Vancomycin &  Rast Pangenome \\
Mean score &  0.976954 &  0.977230 &   0.977930 &  0.976954 &  Random forest &  Vancomycin &  Rast Pangenome \\
std        &  0.006232 &  0.006071 &   0.005821 &  0.006232 &  Random forest &  Vancomycin &  Rast Pangenome \\
\bottomrule
\end{tabular}

van_rast_ML_gb_misclassified = ['SRR13727033', 'SRR13999997', 'SRR14000023', 'SRR14011043',
       'SRR14010988', 'SRR14026552', 'SRR14000586', 'SRR14011017',
       'SRR14000612', 'SRR14011035']
       
van_rast_ML_rf_misclassified = ['SRR14011008', 'SRR14026550', 'SRR14010988', 'SRR14026546',
       'SRR14010973', 'SRR14026521', 'SRR14026552', 'SRR14000586',
       'SRR14011017', 'SRR14026538', 'SRR14000612', 'SRR14026519',
       'SRR14026545', 'SRR14026554']

In [45]:
# July 3rd, 2023

rast_van_intermediate_outcome = van_rast_ML_gb[2].predict(intermediate_Vancomycin)

# prediction completed with the RF like the paper
rast_van_intermediate_outcome_df = pd.DataFrame(list(zip(intermediate_Vancomycin.index.to_list(), rast_van_intermediate_outcome)),
               columns =['Run_accession', 'Intermdiate_pred_rast'])
rast_van_intermediate_outcome_df['Antibiotic'] = 'Vancomycin'
rast_van_intermediate_outcome_df



,Run_accession,Intermdiate_pred_rast,Antibiotic
0,SRR13712503,1,Vancomycin
1,SRR13725688,0,Vancomycin
2,SRR13726511,1,Vancomycin
3,SRR13726526,0,Vancomycin
4,SRR13726563,1,Vancomycin
5,SRR13726564,1,Vancomycin
6,SRR13726569,1,Vancomycin
7,SRR13726576,0,Vancomycin
8,SRR13726577,1,Vancomycin
9,SRR13726578,1,Vancomycin


In [44]:
van_rast_ML_gb[2].predict(intermediate_Vancomycin)

array([1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0,
       0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1])

---

In [26]:
# March 8th, 2023

dox_gb_rast_model = ML_features(Xy_Doxycycline.iloc[:,0:34818],Xy_Doxycycline[['Doxycycline']], gb_clf)
eryth_gb_rast_model = ML_features(Xy_Erythromycin.iloc[:,0:34818],Xy_Erythromycin[['Erythromycin']], gb_clf)

dox_rf_rast_model = ML_features(Xy_Doxycycline.iloc[:,0:34818],Xy_Doxycycline[['Doxycycline']], rf_clf)
eryth_rf_rast_model = ML_features(Xy_Erythromycin.iloc[:,0:34818],Xy_Erythromycin[['Erythromycin']], rf_clf)


/home/jee/conda/envs/ml_prediction/lib/python3.7/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/home/jee/conda/envs/ml_prediction/lib/python3.7/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


,features,weights
1916,group_5465,0.504364
12030,group_24340,0.028960
8313,group_5976,0.027633
8396,group_9448,0.019448
15584,group_7483,0.019029
...,...,...
11682,group_22202,0.000000
11681,group_22058,0.000000
11680,group_21852,0.000000
11679,group_2175,0.000000


/home/jee/conda/envs/ml_prediction/lib/python3.7/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/home/jee/conda/envs/ml_prediction/lib/python3.7/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


,features,weights
1916,group_5465,0.472236
20983,group_19823,0.027629
24203,group_1282,0.024154
10093,ABC-F type ribosomal protection protein > OptrA,0.019971
3259,"lipoprotein, NLP/P60 family",0.018538
...,...,...
11680,group_21852,0.000000
11679,group_2175,0.000000
11678,group_21494,0.000000
11677,group_21439,0.000000


/home/jee/conda/envs/ml_prediction/lib/python3.7/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/home/jee/conda/envs/ml_prediction/lib/python3.7/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


...

/home/jee/conda/envs/ml_prediction/lib/python3.7/site-packages/ipykernel_launcher.py:32: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
/home/jee/conda/envs/ml_prediction/lib/python3.7/site-packages/ipykernel_launcher.py:36: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


,features,weights
11413,group_23508,0.002931
12774,group_23516,0.002918
9923,"ABC-type multidrug transport system, permease ...",0.002917
10714,group_30302,0.002664
10722,group_30310,0.002608
...,...,...
14240,group_19267,0.000000
14239,group_1926,0.000000
14238,group_19235,0.000000
14237,group_19179,0.000000


/home/jee/conda/envs/ml_prediction/lib/python3.7/site-packages/ipykernel_launcher.py:32: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
/home/jee/conda/envs/ml_prediction/lib/python3.7/site-packages/ipykernel_launcher.py:36: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


,features,weights
10711,group_30298,0.002512
11104,FIG00630998: hypothetical protein,0.002270
11158,group_31009,0.002256
11558,Radical SAM domain protein,0.002192
15887,group_1717,0.002060
...,...,...
14556,Eps10K,0.000000
14555,group_29971,0.000000
14553,group_29969,0.000000
14552,group_29968,0.000000


Erythromycin resistance prediction,
   Accuracy        F1  Precision    Recall
0  0.957895  0.956037   0.956290  0.957895
1  0.894737  0.879029   0.879361  0.894737
2  0.905263  0.899337   0.896940  0.905263
3  0.915789  0.908359   0.908167  0.915789
4  0.915789  0.896673   0.923193  0.915789

The misclassified isolates for Erythromycin resistance prediction is stated in a below list:
Index(['SRR13712376', 'SRR13725723', 'SRR13726503', 'SRR13727013',
       'SRR13712375', 'SRR13726527', 'SRR13999956', 'SRR13999978',
       'SRR14000003', 'SRR14026445', 'SRR14026466', 'SRR14026479',
       'SRR14026538', 'SRR14026546', 'SRR13726579', 'SRR13727029',
       'SRR13727035', 'SRR13999921', 'SRR13999932', 'SRR13999984',
       'SRR14022756', 'SRR14026472', 'SRR14026526', 'SRR13712362',
       'SRR13726574', 'SRR13726580', 'SRR14000016', 'SRR14000018',
       'SRR14022743', 'SRR14022776', 'SRR14022782', 'SRR13712368',
       'SRR13712379', 'SRR13999965', 'SRR13999974', 'SRR14022740',
       'S

In [48]:
# July 3rd, 2023

rast_dox_intermediate_outcome = dox_gb_rast_model[2].predict(intermediate_Doxycycline)
rast_erth_intermediate_outcome = eryth_rf_rast_model[2].predict(intermediate_Erythromycin)

# prediction completed with the gb for doxy like the paper
rast_dox_intermediate_outcome_df = pd.DataFrame(list(zip(intermediate_Doxycycline.index.to_list(), rast_dox_intermediate_outcome)),
               columns =['Run_accession', 'Intermdiate_pred_rast'])
rast_dox_intermediate_outcome_df['Antibiotic'] = 'Doxycycline'


# prediction completed with the rf for erth like the paper
rast_erth_intermediate_outcome_df = pd.DataFrame(list(zip(intermediate_Erythromycin.index.to_list(), rast_erth_intermediate_outcome)),
               columns =['Run_accession', 'Intermdiate_pred_rast'])
rast_erth_intermediate_outcome_df['Antibiotic'] = 'Erythromycin'

display(rast_dox_intermediate_outcome_df)
display(rast_erth_intermediate_outcome_df)


,Run_accession,Intermdiate_pred_rast,Antibiotic
0,SRR13712492,1,Doxycycline
1,SRR13712497,1,Doxycycline
2,SRR13725686,1,Doxycycline
3,SRR13725694,1,Doxycycline
4,SRR13725699,0,Doxycycline
...,...,...,...
115,SRR14026505,0,Doxycycline
116,SRR14026515,1,Doxycycline
117,SRR14026535,1,Doxycycline
118,SRR14026538,1,Doxycycline


,Run_accession,Intermdiate_pred_rast,Antibiotic
0,SRR13712369,1,Erythromycin
1,SRR13712371,1,Erythromycin
2,SRR13712372,1,Erythromycin
3,SRR13712378,1,Erythromycin
4,SRR13712381,1,Erythromycin
...,...,...,...
167,SRR14026541,1,Erythromycin
168,SRR14026544,1,Erythromycin
169,SRR14026548,1,Erythromycin
170,SRR14026552,1,Erythromycin


In [54]:
# July 3rd, 2023
# rast_dox_intermediate_outcome_df
# rast_erth_intermediate_outcome_df
dox_intermediate_pred_df = pd.read_csv('/home/jee/Manuscript1_Oct2020/doxy_intermediate_pred_df.csv')
pd.merge(rast_dox_intermediate_outcome_df, dox_intermediate_pred_df, on=['Run_accession','Antibiotic'])#.to_csv('/home/jee/Manuscript1_Oct2020/dox_intermediate_pred_df.csv', index=None)

erth_intermediate_pred_df = pd.read_csv('/home/jee/Manuscript1_Oct2020/erth_intermediate_pred_df.csv')
# pd.merge(rast_erth_intermediate_outcome_df, erth_intermediate_pred_df, on=['Run_accession']).to_csv('/home/jee/Manuscript1_Oct2020/erth_intermediate_pred_df.csv', index=None)

In [25]:
# March 8th, 2023 

dox_rast_ML_gb_df = dox_gb_rast_model[0]
dox_rast_ML_gb_df.loc['Mean score'] = dox_rast_ML_gb_df.mean()
dox_rast_ML_gb_df.loc['std'] = dox_rast_ML_gb_df.std()
dox_rast_ML_gb_df['Classifier'] = 'Gradient boosting'
dox_rast_ML_gb_df['Antibiotic'] = 'Doxycycline'
dox_rast_ML_gb_df['Features'] = 'Rast Pangenome'

dox_rast_ML_rf_df = dox_rf_rast_model[0]
dox_rast_ML_rf_df.loc['Mean score'] = dox_rast_ML_rf_df.mean()
dox_rast_ML_rf_df.loc['std'] = dox_rast_ML_rf_df.std()
dox_rast_ML_rf_df['Classifier'] = 'Random forest'
dox_rast_ML_rf_df['Antibiotic'] = 'Doxycycline'
dox_rast_ML_rf_df['Features'] = 'Rast Pangenome'

eryth_rast_ML_gb_df = eryth_gb_rast_model[0]
eryth_rast_ML_gb_df.loc['Mean score'] = eryth_rast_ML_gb_df.mean()
eryth_rast_ML_gb_df.loc['std'] = eryth_rast_ML_gb_df.std()
eryth_rast_ML_gb_df['Classifier'] = 'Gradient boosting'
eryth_rast_ML_gb_df['Antibiotic'] = 'Erythromycin'
eryth_rast_ML_gb_df['Features'] = 'Rast Pangenome'

eryth_rast_ML_rf_df = eryth_rf_rast_model[0]
eryth_rast_ML_rf_df.loc['Mean score'] = eryth_rast_ML_rf_df.mean()
eryth_rast_ML_rf_df.loc['std'] = eryth_rast_ML_rf_df.std()
eryth_rast_ML_rf_df['Classifier'] = 'Random forest'
eryth_rast_ML_rf_df['Antibiotic'] = 'Erythromycin'
eryth_rast_ML_rf_df['Features'] = 'Rast Pangenome'


In [25]:
# March 8th, 2023

display(dox_rast_ML_gb_df)
display(dox_rast_ML_rf_df)
display(eryth_rast_ML_gb_df)
display(eryth_rast_ML_rf_df)
# Gradient boosting	 & Vancomycin, Doxycycline
# Random forest	Erythromycin
van_gb_ML_df

,Accuracy,F1,Precision,Recall,Classifier,Antibiotic,Features
0,0.915094,0.914353,0.917873,0.915094,Gradient boosting,Doxycycline,Rast Pangenome
1,0.915094,0.915178,0.915426,0.915094,Gradient boosting,Doxycycline,Rast Pangenome
2,0.933333,0.933407,0.933651,0.933333,Gradient boosting,Doxycycline,Rast Pangenome
3,0.933333,0.933036,0.934309,0.933333,Gradient boosting,Doxycycline,Rast Pangenome
4,0.904762,0.904497,0.904904,0.904762,Gradient boosting,Doxycycline,Rast Pangenome
Mean score,0.920323,0.920094,0.921233,0.920323,Gradient boosting,Doxycycline,Rast Pangenome
std,0.011273,0.011359,0.011286,0.011273,Gradient boosting,Doxycycline,Rast Pangenome


,Accuracy,F1,Precision,Recall,Classifier,Antibiotic,Features
0,0.933962,0.933872,0.933952,0.933962,Random forest,Doxycycline,Rast Pangenome
1,0.905660,0.905830,0.906647,0.905660,Random forest,Doxycycline,Rast Pangenome
2,0.895238,0.895638,0.903995,0.895238,Random forest,Doxycycline,Rast Pangenome
3,0.923810,0.923327,0.925624,0.923810,Random forest,Doxycycline,Rast Pangenome
4,0.876190,0.876030,0.876032,0.876190,Random forest,Doxycycline,Rast Pangenome
Mean score,0.906972,0.906940,0.909250,0.906972,Random forest,Doxycycline,Rast Pangenome
std,0.020488,0.020387,0.020080,0.020488,Random forest,Doxycycline,Rast Pangenome


,Accuracy,F1,Precision,Recall,Classifier,Antibiotic,Features
0,0.936842,0.934056,0.933143,0.936842,Gradient boosting,Erythromycin,Rast Pangenome
1,0.894737,0.879029,0.879361,0.894737,Gradient boosting,Erythromycin,Rast Pangenome
2,0.905263,0.899337,0.896940,0.905263,Gradient boosting,Erythromycin,Rast Pangenome
3,0.894737,0.885449,0.882335,0.894737,Gradient boosting,Erythromycin,Rast Pangenome
4,0.894737,0.885449,0.882335,0.894737,Gradient boosting,Erythromycin,Rast Pangenome
Mean score,0.905263,0.896664,0.894823,0.905263,Gradient boosting,Erythromycin,Rast Pangenome
std,0.016307,0.019839,0.020119,0.016307,Gradient boosting,Erythromycin,Rast Pangenome


,Accuracy,F1,Precision,Recall,Classifier,Antibiotic,Features
0,0.947368,0.943675,0.944873,0.947368,Random forest,Erythromycin,Rast Pangenome
1,0.905263,0.894183,0.894412,0.905263,Random forest,Erythromycin,Rast Pangenome
2,0.905263,0.899337,0.896940,0.905263,Random forest,Erythromycin,Rast Pangenome
3,0.905263,0.894183,0.894412,0.905263,Random forest,Erythromycin,Rast Pangenome
4,0.915789,0.896673,0.923193,0.915789,Random forest,Erythromycin,Rast Pangenome
Mean score,0.915789,0.905610,0.910766,0.915789,Random forest,Erythromycin,Rast Pangenome
std,0.016307,0.019128,0.020218,0.016307,Random forest,Erythromycin,Rast Pangenome


In [27]:
# March 29th, 2023

print(dox_rast_ML_gb_df.to_latex())
print(dox_rast_ML_rf_df.to_latex())
print(eryth_rast_ML_gb_df.to_latex())
print(eryth_rast_ML_rf_df.to_latex())

\begin{tabular}{lrrrrlll}
\toprule
{} &  Accuracy &        F1 &  Precision &    Recall &         Classifier &   Antibiotic &        Features \\
\midrule
0          &  0.924528 &  0.923693 &   0.929117 &  0.924528 &  Gradient boosting &  Doxycycline &  Rast Pangenome \\
1          &  0.915094 &  0.915178 &   0.915426 &  0.915094 &  Gradient boosting &  Doxycycline &  Rast Pangenome \\
2          &  0.923810 &  0.923963 &   0.924791 &  0.923810 &  Gradient boosting &  Doxycycline &  Rast Pangenome \\
3          &  0.933333 &  0.933036 &   0.934309 &  0.933333 &  Gradient boosting &  Doxycycline &  Rast Pangenome \\
4          &  0.914286 &  0.914175 &   0.914233 &  0.914286 &  Gradient boosting &  Doxycycline &  Rast Pangenome \\
Mean score &  0.922210 &  0.922009 &   0.923575 &  0.922210 &  Gradient boosting &  Doxycycline &  Rast Pangenome \\
std        &  0.007001 &  0.006874 &   0.007760 &  0.007001 &  Gradient boosting &  Doxycycline &  Rast Pangenome \\
\bottomrule
\end{tabular}

\

In [30]:
display(dox_gb_rast_model[3])
display(eryth_rf_rast_model[3])

Index(['SRR13725703', 'SRR13727030', 'SRR13999961', 'SRR14022745',
       'SRR14024950', 'SRR14024964', 'SRR14024991', 'SRR14026500',
       'SRR13725689', 'SRR13726527', 'SRR13727011', 'SRR13999957',
       'SRR14000595', 'SRR14010969', 'SRR14010976', 'SRR14011044',
       'SRR14024957', 'SRR13727015', 'SRR14000007', 'SRR14000025',
       'SRR14000598', 'SRR14000614', 'SRR14011009', 'SRR14022771',
       'SRR14024961', 'SRR13727005', 'SRR13999948', 'SRR14000003',
       'SRR14010992', 'SRR14010996', 'SRR14024956', 'SRR14026488',
       'SRR13712365', 'SRR13725696', 'SRR13727008', 'SRR13727028',
       'SRR13999978', 'SRR13999986', 'SRR13999990', 'SRR14010955',
       'SRR14011038'],
      dtype='object', name='GenomeID')

Index(['SRR13712366', 'SRR13726503', 'SRR13726539', 'SRR13727013',
       'SRR14022758', 'SRR13712375', 'SRR13726527', 'SRR13999956',
       'SRR13999978', 'SRR14026445', 'SRR14026466', 'SRR14026479',
       'SRR14026538', 'SRR14026546', 'SRR13726579', 'SRR13727029',
       'SRR13727035', 'SRR13999921', 'SRR13999932', 'SRR13999984',
       'SRR14022756', 'SRR14026472', 'SRR14026526', 'SRR13712362',
       'SRR13726574', 'SRR13726580', 'SRR14000016', 'SRR14000018',
       'SRR14022743', 'SRR14022776', 'SRR14022782', 'SRR13712368',
       'SRR13999965', 'SRR13999974', 'SRR14022740', 'SRR14022750',
       'SRR14026454', 'SRR14026545'],
      dtype='object', name='GenomeID')

**Results; Doxycycline - Gradient boosting; Erythromycin - Random forest**

\begin{tabular}{lrrrrlll}
\toprule
{} &  Accuracy &        F1 &  Precision &    Recall &         Classifier &   Antibiotic &        Features \\
\midrule
0          &  0.924528 &  0.923693 &   0.929117 &  0.924528 &  Gradient boosting &  Doxycycline &  Rast Pangenome \\
1          &  0.915094 &  0.915178 &   0.915426 &  0.915094 &  Gradient boosting &  Doxycycline &  Rast Pangenome \\
2          &  0.923810 &  0.923963 &   0.924791 &  0.923810 &  Gradient boosting &  Doxycycline &  Rast Pangenome \\
3          &  0.933333 &  0.933036 &   0.934309 &  0.933333 &  Gradient boosting &  Doxycycline &  Rast Pangenome \\
4          &  0.914286 &  0.914175 &   0.914233 &  0.914286 &  Gradient boosting &  Doxycycline &  Rast Pangenome \\
Mean score &  0.922210 &  0.922009 &   0.923575 &  0.922210 &  Gradient boosting &  Doxycycline &  Rast Pangenome \\
std        &  0.007001 &  0.006874 &   0.007760 &  0.007001 &  Gradient boosting &  Doxycycline &  Rast Pangenome \\
\bottomrule
\end{tabular}

\begin{tabular}{lrrrrlll}
\toprule
{} &  Accuracy &        F1 &  Precision &    Recall &     Classifier &   Antibiotic &        Features \\
\midrule
0          &  0.943396 &  0.943396 &   0.943396 &  0.943396 &  Random forest &  Doxycycline &  Rast Pangenome \\
1          &  0.905660 &  0.905830 &   0.906647 &  0.905660 &  Random forest &  Doxycycline &  Rast Pangenome \\
2          &  0.904762 &  0.905126 &   0.911397 &  0.904762 &  Random forest &  Doxycycline &  Rast Pangenome \\
3          &  0.933333 &  0.933036 &   0.934309 &  0.933333 &  Random forest &  Doxycycline &  Rast Pangenome \\
4          &  0.876190 &  0.876030 &   0.876032 &  0.876190 &  Random forest &  Doxycycline &  Rast Pangenome \\
Mean score &  0.912668 &  0.912684 &   0.914356 &  0.912668 &  Random forest &  Doxycycline &  Rast Pangenome \\
std        &  0.023721 &  0.023686 &   0.023573 &  0.023721 &  Random forest &  Doxycycline &  Rast Pangenome \\
\bottomrule
\end{tabular}

\begin{tabular}{lrrrrlll}
\toprule
{} &  Accuracy &        F1 &  Precision &    Recall &         Classifier &    Antibiotic &        Features \\
\midrule
0          &  0.926316 &  0.924779 &   0.923653 &  0.926316 &  Gradient boosting &  Erythromycin &  Rast Pangenome \\
1          &  0.894737 &  0.879029 &   0.879361 &  0.894737 &  Gradient boosting &  Erythromycin &  Rast Pangenome \\
2          &  0.905263 &  0.899337 &   0.896940 &  0.905263 &  Gradient boosting &  Erythromycin &  Rast Pangenome \\
3          &  0.894737 &  0.885449 &   0.882335 &  0.894737 &  Gradient boosting &  Erythromycin &  Rast Pangenome \\
4          &  0.894737 &  0.885449 &   0.882335 &  0.894737 &  Gradient boosting &  Erythromycin &  Rast Pangenome \\
Mean score &  0.903158 &  0.894809 &   0.892925 &  0.903158 &  Gradient boosting &  Erythromycin &  Rast Pangenome \\
std        &  0.012276 &  0.016389 &   0.016545 &  0.012276 &  Gradient boosting &  Erythromycin &  Rast Pangenome \\
\bottomrule
\end{tabular}

\begin{tabular}{lrrrrlll}
\toprule
{} &  Accuracy &        F1 &  Precision &    Recall &     Classifier &    Antibiotic &        Features \\
\midrule
0          &  0.947368 &  0.946271 &   0.945635 &  0.947368 &  Random forest &  Erythromycin &  Rast Pangenome \\
1          &  0.905263 &  0.894183 &   0.894412 &  0.905263 &  Random forest &  Erythromycin &  Rast Pangenome \\
2          &  0.905263 &  0.899337 &   0.896940 &  0.905263 &  Random forest &  Erythromycin &  Rast Pangenome \\
3          &  0.915789 &  0.908359 &   0.908167 &  0.915789 &  Random forest &  Erythromycin &  Rast Pangenome \\
4          &  0.926316 &  0.912636 &   0.932047 &  0.926316 &  Random forest &  Erythromycin &  Rast Pangenome \\
Mean score &  0.920000 &  0.912157 &   0.915440 &  0.920000 &  Random forest &  Erythromycin &  Rast Pangenome \\
std        &  0.015754 &  0.018253 &   0.020124 &  0.015754 &  Random forest &  Erythromycin &  Rast Pangenome \\
\bottomrule
\end{tabular}

dox_gb_rast_model_misclassified = ['SRR13725703', 'SRR13727030', 'SRR13999961', 'SRR14022745',
       'SRR14024950', 'SRR14024964', 'SRR14024991', 'SRR14026500',
       'SRR13725689', 'SRR13726527', 'SRR13727011', 'SRR13999957',
       'SRR14000595', 'SRR14010969', 'SRR14010976', 'SRR14011044',
       'SRR14024957', 'SRR13727015', 'SRR14000007', 'SRR14000025',
       'SRR14000598', 'SRR14000614', 'SRR14011009', 'SRR14022771',
       'SRR14024961', 'SRR13727005', 'SRR13999948', 'SRR14000003',
       'SRR14010992', 'SRR14010996', 'SRR14024956', 'SRR14026488',
       'SRR13712365', 'SRR13725696', 'SRR13727008', 'SRR13727028',
       'SRR13999978', 'SRR13999986', 'SRR13999990', 'SRR14010955',
       'SRR14011038']
       
eryth_rf_rast_model_misclassified = ['SRR13712366', 'SRR13726503', 'SRR13726539', 'SRR13727013',
       'SRR14022758', 'SRR13712375', 'SRR13726527', 'SRR13999956',
       'SRR13999978', 'SRR14026445', 'SRR14026466', 'SRR14026479',
       'SRR14026538', 'SRR14026546', 'SRR13726579', 'SRR13727029',
       'SRR13727035', 'SRR13999921', 'SRR13999932', 'SRR13999984',
       'SRR14022756', 'SRR14026472', 'SRR14026526', 'SRR13712362',
       'SRR13726574', 'SRR13726580', 'SRR14000016', 'SRR14000018',
       'SRR14022743', 'SRR14022776', 'SRR14022782', 'SRR13712368',
       'SRR13999965', 'SRR13999974', 'SRR14022740', 'SRR14022750',
       'SRR14026454', 'SRR14026545']

### HSIC-Lasso for RAST pangenomes

In [28]:
from pyHSICLasso import HSICLasso #https://github.com/riken-aip/pyHSICLasso
hsic_lasso = HSICLasso()

In [190]:
# E. faecium & E. faeaclis - Vancomycin
hsic_lasso.input('/home/jee/Manuscript1_Oct2020/pyHSICLasso/rast_pan/rast_Xy_Vancomycin_HSIC.csv',output_list=['Vancomycin'])
hsic_lasso.classification(10)
hsic_lasso.save_score(filename="/home/jee/Manuscript1_Oct2020/pyHSICLasso/rast_pan/rast_Vancomycin_HSICscore.csv")

Block HSIC Lasso B = 20.
M set to 3.
Using Gaussian kernel for the features, Delta kernel for the outcomes.


/home/jee/conda/envs/ml_prediction/lib/python3.7/site-packages/pyHSICLasso/api.py:107: RuntimeWarning: B 20 must be an exact divisor of the number of samples 608. Number of blocks 30.4 will be approximated to 30.
  warnings.warn(msg, RuntimeWarning)
/home/jee/conda/envs/ml_prediction/lib/python3.7/site-packages/pyHSICLasso/nlars.py:77: RuntimeWarning: invalid value encountered in true_divide
  gamma1 = (C - c[I]) / (XtXw[A[0]] - XtXw[I])
/home/jee/conda/envs/ml_prediction/lib/python3.7/site-packages/pyHSICLasso/nlars.py:77: RuntimeWarning: divide by zero encountered in true_divide
  gamma1 = (C - c[I]) / (XtXw[A[0]] - XtXw[I])


In [191]:
# E. faecium & E. faeaclis - Doxycycline
hsic_lasso.input('/home/jee/Manuscript1_Oct2020/pyHSICLasso/rast_pan/rast_Xy_Doxycycline_HSIC.csv',output_list=['Doxycycline'])
hsic_lasso.classification(10)
hsic_lasso.save_score(filename="/home/jee/Manuscript1_Oct2020/pyHSICLasso/rast_pan/rast_Doxycycline_HSICscore.csv")

Block HSIC Lasso B = 20.
M set to 3.
Using Gaussian kernel for the features, Delta kernel for the outcomes.


/home/jee/conda/envs/ml_prediction/lib/python3.7/site-packages/pyHSICLasso/api.py:107: RuntimeWarning: B 20 must be an exact divisor of the number of samples 527. Number of blocks 26.35 will be approximated to 26.
  warnings.warn(msg, RuntimeWarning)
/home/jee/conda/envs/ml_prediction/lib/python3.7/site-packages/pyHSICLasso/nlars.py:78: RuntimeWarning: divide by zero encountered in true_divide
  gamma2 = -beta[A] / (w)


In [29]:
# E. faecium & E. faeaclis - Erythromycin
hsic_lasso.input('/home/jee/Manuscript1_Oct2020/pyHSICLasso/rast_pan/rast_Xy_Erythromycin_HSIC.csv',output_list=['Erythromycin'])
hsic_lasso.classification(10)
hsic_lasso.save_score(filename="/home/jee/Manuscript1_Oct2020/pyHSICLasso/rast_pan/rast_Erythromycin_HSICscore.csv")

Block HSIC Lasso B = 20.
M set to 3.
Using Gaussian kernel for the features, Delta kernel for the outcomes.


In [60]:
df_hsic_van = pd.read_csv('/home/jee/Manuscript1_Oct2020/pyHSICLasso/rast_pan/rast_Vancomycin_HSICscore.csv').iloc[0:50]
df_hsic_van['Antibiotic'] = 'Vancomycin'
df_hsic_van                          

,Feature,Score,Pearson Corr,Antibiotic
0,D-alanyl-D-alanine dipeptidase (EC 3.4.13.22) ...,1.292218,0.970681,Vancomycin
1,D-lactate dehydrogenase VanH; associated with ...,1.270782,0.970681,Vancomycin
2,D-alanine--(R)-lactate ligase (EC 6.1.2.1) > VanA,1.270384,0.970681,Vancomycin
3,Vancomycin (or other glycopeptides) response r...,1.255141,0.964449,Vancomycin
4,group_18897,1.099156,0.885429,Vancomycin
5,group_18895,1.091344,0.880315,Vancomycin
6,group_18896,1.091344,0.880315,Vancomycin
7,FIG00628452: hypothetical protein,1.091344,0.880315,Vancomycin
8,Putrescine transport ATP-binding protein PotA ...,1.091344,0.880315,Vancomycin
9,FIG00632940: hypothetical protein,0.852434,0.873912,Vancomycin


In [61]:
df_hsic_dox = pd.read_csv('/home/jee/Manuscript1_Oct2020/pyHSICLasso/rast_pan/rast_Doxycycline_HSICscore.csv', sep=',').iloc[0:50]
df_hsic_dox['Antibiotic'] = 'Doxycycline'
df_hsic_dox

,Feature,Score,Pearson Corr,Antibiotic
0,group_5465,1.279960,0.783804,Doxycycline
1,FIG00627334: hypothetical protein,1.127100,0.696995,Doxycycline
2,lipoprotein; NLP/P60 family,1.120794,0.730075,Doxycycline
3,Antirestriction protein,1.110554,0.706840,Doxycycline
4,Tn916; hypothetical protein,1.104672,0.729702,Doxycycline
5,FIG086557: Conjugation related protein,1.104672,0.729702,Doxycycline
6,Tn916; transcriptional regulator-putative,1.094666,0.725879,Doxycycline
7,FIG00627241: hypothetical protein,1.053453,0.716525,Doxycycline
8,FtsK/SpoIIIE family protein,1.036747,0.710267,Doxycycline
9,group_13100,1.024490,0.710267,Doxycycline


In [62]:
df_hsic_eryth = pd.read_csv('/home/jee/Manuscript1_Oct2020/pyHSICLasso/rast_pan/rast_Erythromycin_HSICscore.csv', sep=',').iloc[0:50]
df_hsic_eryth['Antibiotic'] = 'Erythromycin'
df_hsic_eryth

,Feature,Score,Pearson Corr,Antibiotic
0,group_14181,1.000000,-0.348980,Erythromycin
1,group_11178,0.840281,-0.321120,Erythromycin
2,group_14199,0.826937,-0.366011,Erythromycin
3,group_5556,0.794544,-0.329139,Erythromycin
4,group_8336,0.712244,-0.313983,Erythromycin
5,group_5824,0.558105,-0.316103,Erythromycin
6,group_19471,0.506637,-0.340016,Erythromycin
7,group_16417,0.483091,-0.321062,Erythromycin
8,group_29945,0.483091,-0.321062,Erythromycin
9,group_29962,0.483091,-0.321062,Erythromycin


In [67]:
hsic_ab_feat = pd.concat([df_hsic_van, df_hsic_dox, df_hsic_eryth])

#### Feat ALL

In [31]:
# hsic_rastfeatimp = all_ab_feat.merge(hsic_ab_feat, how='outer', on=['Feature','Antibiotic'])[['Feature','weights','Score','Pearson Corr','Antibiotic']]
pd.read_csv('/home/jee/bvbrc_workshop_Aug2022/HSIC_rastfeatimport.csv')

,Feature,weights,Score,Pearson Corr,Antibiotic
0,"D-lactate dehydrogenase VanH, associated with ...","[0.4035662260885783, 0.2006093073478255, 0.389...",NaN,NaN,Vancomycin
1,D-alanyl-D-alanine dipeptidase (EC 3.4.13.22) ...,"[0.2737536911759534, 0.4950966822906833, 0.390...",1.292218,0.970681,Vancomycin
2,D-alanine--(R)-lactate ligase (EC 6.1.2.1) > VanA,"[0.2510646099185377, 0.2459126660542203, 0.161...",1.270384,0.970681,Vancomycin
3,group_5202,[0.018162213417159227],NaN,NaN,Vancomycin
4,group_3602,[0.018153109297851414],NaN,NaN,Vancomycin
...,...,...,...,...,...
82,group_29962,NaN,0.483091,-0.321062,Erythromycin
83,regulatory protein; ArsR,NaN,0.468638,-0.353726,Erythromycin
84,group_16549,NaN,0.456024,-0.331357,Erythromycin
85,group_30352,NaN,0.456023,-0.342202,Erythromycin


The above two DFs are saved as one excel file (two sheets) in my latop as: <br>
**"C:\Users\jeein\Google Drive\dalhousie\Publications\Manuscript1\rast_feat_results.csv"**

### Feature importance ranking
March 8th, 2023 

* Vancomycin: Gradient boosting 
* Doxycycline: Gradient boosting
* Erythromycin: Random forest

In [82]:
from functools import reduce
'''ml_model_featimportance = [ van_rast_ML_gb,
                               dox_gb_rast_model,
                               eryth_rf_rast_model ]'''

    
def feat_ranking(ml_model_featimportance):

# for each cross-fold, new dataframes with feature importance ranking  
    ranking_ls = []

    for i in range(5):
        ml_model_featimportance[1][i]['Ranking'] = ml_model_featimportance[1][i].reset_index().index + 1
        ranking_ls.append(ml_model_featimportance[1][i].rename(columns={'features':'Feature'}))

    dfs = [ranking_ls[0].rename(columns={'Ranking':'Ranking_1'})[['Feature','Ranking_1']],
                         ranking_ls[1].rename(columns={'Ranking':'Ranking_2'})[['Feature','Ranking_2']],
                         ranking_ls[2].rename(columns={'Ranking':'Ranking_3'})[['Feature','Ranking_3']],
                         ranking_ls[3].rename(columns={'Ranking':'Ranking_4'})[['Feature','Ranking_4']],
                         ranking_ls[4].rename(columns={'Ranking':'Ranking_5'})[['Feature','Ranking_5']] 
          ]
    
    merged_df = reduce(lambda  left,right: pd.merge(left,right,on=['Feature'], how='outer'), dfs)
    
    merged_df['Avg feat importance ranking'] = merged_df.iloc[:,3:8].mean(axis = 1).apply(lambda x: float("{:.2f}".format(x)))

    return merged_df
    


In [33]:
van_rast_ML_gb_feat_ranking = feat_ranking(van_rast_ML_gb)
dox_rast_ML_gb_feat_ranking = feat_ranking(dox_gb_rast_model)
eryth_rast_ML_rf_feat_ranking = feat_ranking(eryth_rf_rast_model)

In [35]:
van_rast_ML_gb_feat_ranking['Antibiotic'] = 'Vancomycin'
dox_rast_ML_gb_feat_ranking['Antibiotic'] = 'Doxycycline'
eryth_rast_ML_rf_feat_ranking['Antibiotic'] = 'Erythromycin'

In [37]:
van_rast_ML_gb_feat_ranking['classifier'] = 'gradient boosting'
dox_rast_ML_gb_feat_ranking['classifier'] = 'gradient boosting'
eryth_rast_ML_rf_feat_ranking['classifier'] = 'random forest'

In [39]:
rast_feat_ranking = pd.concat([van_rast_ML_gb_feat_ranking.iloc[0:50,:],
dox_rast_ML_gb_feat_ranking.iloc[0:50,:],
eryth_rast_ML_rf_feat_ranking.iloc[0:50,:]])

#rast_feat_ranking.to_csv('/home/jee/Manuscript1_Oct2020/rast_feat_ranking_March2023.csv', index=None)

In [42]:
rast_feat_ranking = pd.read_csv('/home/jee/Manuscript1_Oct2020/rast_feat_ranking_March2023.csv')

,Feature,Ranking_1,Ranking_2,Ranking_3,Ranking_4,Ranking_5,Avg feat importance ranking,Antibiotic,classifier
0,D-alanyl-D-alanine dipeptidase (EC 3.4.13.22) ...,1,1,2,3,1,2.00,Vancomycin,gradient boosting
1,D-alanine--(R)-lactate ligase (EC 6.1.2.1) > VanA,2,3,1,1,3,1.67,Vancomycin,gradient boosting
2,"D-lactate dehydrogenase VanH, associated with ...",3,2,3,2,2,2.33,Vancomycin,gradient boosting
3,group_3602,4,28612,28788,28615,28601,28668.00,Vancomycin,gradient boosting
4,group_9855,5,24775,6,5,24796,8269.00,Vancomycin,gradient boosting
...,...,...,...,...,...,...,...,...,...
45,group_3831,46,31882,617,102,450,389.67,Erythromycin,random forest
46,group_30340,47,95,414,289,107,270.00,Erythromycin,random forest
47,group_30298,48,57,53,29,7,29.67,Erythromycin,random forest
48,group_19520,49,40,297,40,226,187.67,Erythromycin,random forest


### Features annotated again against GenBank

In [96]:
from Bio import SeqIO 
from Bio.SeqRecord import SeqRecord

# path: '/home/jee/bvbrc_workshop_Aug2022/bvbrc_annot/rast_gff/roary_output/pan_genome_reference.fa'

# abx_flist = list of genes that are selected by number of feature selection methods
# abx = antibiotic

def selected_feat_seq(abx_flist, abx):
    #load the pangenome
    fa = SeqIO.parse("/home/jee/bvbrc_workshop_Aug2022/bvbrc_annot/rast_gff/roary_output/pan_genome_reference.fa", "fasta")

    #Make a list for the records youre keeping
    records = []

    #Suppose that the genes are in a list
    my_genes = abx_flist

    #not_found = set(my_genes)

    #iterate the records 
    for record in fa:
        # The gene name would be "description"
        if record.description.split(' ',1)[-1] in my_genes:
            records.append(record)
            #not_found.remove(record.description)

    for i,record in enumerate(records):
            #print('initial record id:', record.id)            
            if record.id == record.description.split(' ')[0]: record.id = record.description.split(' ')[-1]
            record.description = record.description.split(' ')[-1]
            print(i,'corrected record description:', record.description)

    # Write the list of records to a new file
    with open("{ab}_feat.fasta".format(ab=abx), "w") as f:
        SeqIO.write(records, f, "fasta")

In [97]:
#pangenome reference file obtained via Roary software: 
fa = SeqIO.parse("/home/jee/bvbrc_workshop_Aug2022/bvbrc_annot/rast_gff/roary_output/pan_genome_reference.fa", "fasta")


In [78]:
van_rast_blast = list(hsic_ab_feat[hsic_ab_feat['Antibiotic']=='Vancomycin'].Feature.unique()) + list(rast_feat_ranking[rast_feat_ranking['Antibiotic']=='Vancomycin'].Feature.unique())
dox_rast_blast = list(hsic_ab_feat[hsic_ab_feat['Antibiotic']=='Doxycycline'].Feature.unique()) + list(rast_feat_ranking[rast_feat_ranking['Antibiotic']=='Doxycycline'].Feature.unique())
eryth_rast_blast = list(hsic_ab_feat[hsic_ab_feat['Antibiotic']=='Erythromycin'].Feature.unique()) + list(rast_feat_ranking[rast_feat_ranking['Antibiotic']=='Erythromycin'].Feature.unique())

In [ ]:
selected_feat_seq(van_rast_blast, 'Vancomycin') #resulted in list of records: 'Vancomycin_feat.fasta'
selected_feat_seq(dox_rast_blast, 'Doxycycline') #resulted in list of records: 'Doxycycline_feat.fasta'
selected_feat_seq(eryth_rast_blast, 'Erythromycin') #resulted in list of records: 'Erythromycin_feat.fasta'

__Bash commands__: <br>
<code> $ for file in *.fasta; do blastn -db nt -query $file -max_target_seqs 5 -outfmt 6 -out $file.blastresults -remote; done </code>

_Above bash command with the initial files of:_<br>
* 'Vancomycin_feat.fasta' --> Vancomycin_feat.fasta.blastresults
* 'Doxycycline_feat.fasta' --> Doxycycline_feat.fasta.blastresults
* 'Erythromycin_feat.fasta' --> Erythromycin_feat.fasta.blastresults

In [95]:
header_list = ['qseqid', 'sseqid', 'pident', 'length', 'mismatch', 'gapopen',
               'qstart', 'qend', 'sstart', 'send', 'evalue', 'bitscore']
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

Vancomycin_rast_feat_results = pd.read_csv('Vancomycin_feat.fasta.blastresults', sep='\t',names=header_list)
Doxycycline_rast_feat_results = pd.read_csv('Doxycycline_feat.fasta.blastresults', sep='\t',names=header_list)
Erythromycin_rast_feat_results = pd.read_csv('Erythromycin_feat.fasta.blastresults', sep='\t',names=header_list)

In [322]:
Vancomycin_rast_feat_results[['sseqid','sstart','send']].to_csv('Vancomycin_rast_for_efetch.tsv',sep='\t',index=None,header=None)
Doxycycline_rast_feat_results[['sseqid','sstart','send']].to_csv('Doxycycline_rast_for_efetch.tsv',sep='\t',index=None,header=None)
Erythromycin_rast_feat_results[['sseqid','sstart','send']].to_csv('Erythromycin_rast_for_efetch.tsv',sep='\t',index=None,header=None)

__Bash commands for NCBI Entrez direct: efetch__ <br>

$ while IFs='' read -r id sstart send; do efetch -db nuccore -format gb -id $id -seq_start $sstart -seq_stop $send \ 
</dev/null; done < Vancomycin_rast_for_efetch.tsv >> Vancomycin_rast_for_efetch.output 

$ while IFs='' read -r id sstart send; do efetch -db nuccore -format gb -id $id -seq_start $sstart -seq_stop $send \
</dev/null; done < Doxycycline_rast_for_efetch.tsv >> Doxycycline_rast_for_efetch.output

$ while IFs='' read -r id sstart send; do efetch -db nuccore -format gb -id $id -seq_start $sstart -seq_stop $send \
</dev/null; done < Erythromycin_rast_for_efetch.tsv >> Erythromycin_rast_for_efetch.output 

In [324]:
Vancomycin_rast_recs = [Vancomycin_rast_recs for Vancomycin_rast_recs in SeqIO.parse('Vancomycin_rast_for_efetch.output', 'genbank')]
Doxycycline_rast_recs = [Doxycycline_rast_recs for Doxycycline_rast_recs in SeqIO.parse('Doxycycline_rast_for_efetch.output', 'genbank')]
Erythromycin_rast_recs = [Erythromycin_rast_recs for Erythromycin_rast_recs in SeqIO.parse('Erythromycin_rast_for_efetch.output', 'genbank')]

gb_dict = {'Vancomycin_rast_recs' : Vancomycin_rast_recs, 
            'Doxycycline_rast_recs' : Doxycycline_rast_recs,
              'Erythromycin_rast_recs': Erythromycin_rast_recs}  

In [326]:
# genbank_parsed_file : ab_recs

def genbankparse(genbank_parsed_file):
    feature_ls = []

    for rec in genbank_parsed_file:
             
        for feat in rec.features:
            
            '''if not "CDS" in feat.type: 
                print('No associated CDS in', rec.annotations["accessions"])'''
            
            if feat.type == "CDS":
                print(rec.id)
                print(rec.annotations["accessions"][2])
                print(rec.description)
                proteinID_ls = ""
                product_ls = ""
                if "protein_id" in feat.qualifiers:
                    print('For protein_id:', feat.qualifiers["protein_id"])
                    proteinID_ls = feat.qualifiers["protein_id"]
                if "product" in feat.qualifiers:
                    print('For products:', feat.qualifiers["product"])
                    product_ls = feat.qualifiers["product"]
            
                '''
                # if I want to exclude proteins that are labelled as 'hypothetical protein'

                if not "hypothetical protein" in feat.qualifiers["product"]:
                    print('For products:', feat.qualifiers["product"])
                    product_ls = feat.qualifiers["product"]
                '''
                
                feature_ls.append([rec.id,rec.annotations["accessions"][2],rec.description,proteinID_ls,product_ls])
        print('######################')
        print()
    return feature_ls


genbankparse(Vancomycin_rast_recs)

######################

######################

CP098418.1
2840420..2840548
Enterococcus faecalis strain CQ025 chromosome, complete genome
For protein_id: ['USA06733.1']
For products: ['WxL domain-containing protein']
######################

CP098418.1
2660696..2660824
Enterococcus faecalis strain CQ025 chromosome, complete genome
For protein_id: ['USA06578.1']
For products: ['WxL domain-containing protein']
######################

CP098025.1
2898079..2898207
Enterococcus faecalis strain QZ076 chromosome, complete genome
For protein_id: ['URQ82903.1']
For products: ['WxL domain-containing protein']
######################

CP098025.1
2718355..2718483
Enterococcus faecalis strain QZ076 chromosome, complete genome
For protein_id: ['URQ82748.1']
For products: ['WxL domain-containing protein']
######################

CP091227.1
2680671..2680799
Enterococcus faecalis strain 661 chromosome, complete genome
For protein_id: ['UQQ70600.1']
For products: ['WxL domain-containing protein']
########

[['CP098418.1',
  '2840420..2840548',
  'Enterococcus faecalis strain CQ025 chromosome, complete genome',
  ['USA06733.1'],
  ['WxL domain-containing protein']],
 ['CP098418.1',
  '2660696..2660824',
  'Enterococcus faecalis strain CQ025 chromosome, complete genome',
  ['USA06578.1'],
  ['WxL domain-containing protein']],
 ['CP098025.1',
  '2898079..2898207',
  'Enterococcus faecalis strain QZ076 chromosome, complete genome',
  ['URQ82903.1'],
  ['WxL domain-containing protein']],
 ['CP098025.1',
  '2718355..2718483',
  'Enterococcus faecalis strain QZ076 chromosome, complete genome',
  ['URQ82748.1'],
  ['WxL domain-containing protein']],
 ['CP091227.1',
  '2680671..2680799',
  'Enterococcus faecalis strain 661 chromosome, complete genome',
  ['UQQ70600.1'],
  ['WxL domain-containing protein']],
 ['CP091227.1',
  '2508283..2508411',
  'Enterococcus faecalis strain 661 chromosome, complete genome',
  ['UQQ70450.1'],
  ['WxL domain-containing protein']],
 ['CP097048.1',
  '2579748..2579

In [327]:
products_ls = []

for key, value in gb_dict.items():
    temp_df = pd.DataFrame(genbankparse(value),columns=["Genome","Region", "Description","Protein ID","Product"])
    temp_df["key"] = f"{key}"
    products_ls.append(temp_df)
    
product_df = pd.concat(products_ls)
product_df = pd.DataFrame(product_df)
product_df

######################

######################

CP098418.1
2840420..2840548
Enterococcus faecalis strain CQ025 chromosome, complete genome
For protein_id: ['USA06733.1']
For products: ['WxL domain-containing protein']
######################

CP098418.1
2660696..2660824
Enterococcus faecalis strain CQ025 chromosome, complete genome
For protein_id: ['USA06578.1']
For products: ['WxL domain-containing protein']
######################

CP098025.1
2898079..2898207
Enterococcus faecalis strain QZ076 chromosome, complete genome
For protein_id: ['URQ82903.1']
For products: ['WxL domain-containing protein']
######################

CP098025.1
2718355..2718483
Enterococcus faecalis strain QZ076 chromosome, complete genome
For protein_id: ['URQ82748.1']
For products: ['WxL domain-containing protein']
######################

CP091227.1
2680671..2680799
Enterococcus faecalis strain 661 chromosome, complete genome
For protein_id: ['UQQ70600.1']
For products: ['WxL domain-containing protein']
########

,Genome,Region,Description,Protein ID,Product,key
0,CP098418.1,2840420..2840548,"Enterococcus faecalis strain CQ025 chromosome, complete genome",[USA06733.1],[WxL domain-containing protein],Vancomycin_rast_recs
1,CP098418.1,2660696..2660824,"Enterococcus faecalis strain CQ025 chromosome, complete genome",[USA06578.1],[WxL domain-containing protein],Vancomycin_rast_recs
2,CP098025.1,2898079..2898207,"Enterococcus faecalis strain QZ076 chromosome, complete genome",[URQ82903.1],[WxL domain-containing protein],Vancomycin_rast_recs
3,CP098025.1,2718355..2718483,"Enterococcus faecalis strain QZ076 chromosome, complete genome",[URQ82748.1],[WxL domain-containing protein],Vancomycin_rast_recs
4,CP091227.1,2680671..2680799,"Enterococcus faecalis strain 661 chromosome, complete genome",[UQQ70600.1],[WxL domain-containing protein],Vancomycin_rast_recs
5,CP091227.1,2508283..2508411,"Enterococcus faecalis strain 661 chromosome, complete genome",[UQQ70450.1],[WxL domain-containing protein],Vancomycin_rast_recs
6,CP097048.1,2579748..2579876,"Enterococcus faecalis strain AT22 chromosome, complete genome",[UQF50965.1],[WxL domain-containing protein],Vancomycin_rast_recs
7,CP097048.1,2761298..2761426,"Enterococcus faecalis strain AT22 chromosome, complete genome",[UQF51120.1],[WxL domain-containing protein],Vancomycin_rast_recs
8,CP103863.1,complement(691962..692087),"Enterococcus faecalis strain SJ82 chromosome, complete genome",[UWE01447.1],[DgaE family pyridoxal phosphate-dependent ammonia lyase],Vancomycin_rast_recs
9,CP103863.1,1778651..1778776,"Enterococcus faecalis strain SJ82 chromosome, complete genome",[UWD99825.1],[DgaE family pyridoxal phosphate-dependent ammonia lyase],Vancomycin_rast_recs


In [328]:
product_df['Product'] = product_df['Product'].astype(str).str.strip("[]").str.strip("''")
product_df['Protein ID'] = product_df['Protein ID'].astype(str).str.strip("[]").str.strip("''")
# product_df.to_csv('rast_fs_products.csv')

In [100]:
pd.read_csv('/home/jee/bvbrc_workshop_Aug2022/rast_fs_products.csv')

,Unnamed: 0,Genome,Region,Description,Protein ID,Product,key
0,0,CP098418.1,2840420..2840548,"Enterococcus faecalis strain CQ025 chromosome, complete genome",USA06733.1,WxL domain-containing protein,Vancomycin_rast_recs
1,1,CP098418.1,2660696..2660824,"Enterococcus faecalis strain CQ025 chromosome, complete genome",USA06578.1,WxL domain-containing protein,Vancomycin_rast_recs
2,2,CP098025.1,2898079..2898207,"Enterococcus faecalis strain QZ076 chromosome, complete genome",URQ82903.1,WxL domain-containing protein,Vancomycin_rast_recs
3,3,CP098025.1,2718355..2718483,"Enterococcus faecalis strain QZ076 chromosome, complete genome",URQ82748.1,WxL domain-containing protein,Vancomycin_rast_recs
4,4,CP091227.1,2680671..2680799,"Enterococcus faecalis strain 661 chromosome, complete genome",UQQ70600.1,WxL domain-containing protein,Vancomycin_rast_recs
5,5,CP091227.1,2508283..2508411,"Enterococcus faecalis strain 661 chromosome, complete genome",UQQ70450.1,WxL domain-containing protein,Vancomycin_rast_recs
6,6,CP097048.1,2579748..2579876,"Enterococcus faecalis strain AT22 chromosome, complete genome",UQF50965.1,WxL domain-containing protein,Vancomycin_rast_recs
7,7,CP097048.1,2761298..2761426,"Enterococcus faecalis strain AT22 chromosome, complete genome",UQF51120.1,WxL domain-containing protein,Vancomycin_rast_recs
8,8,CP103863.1,complement(691962..692087),"Enterococcus faecalis strain SJ82 chromosome, complete genome",UWE01447.1,DgaE family pyridoxal phosphate-dependent ammonia lyase,Vancomycin_rast_recs
9,9,CP103863.1,1778651..1778776,"Enterococcus faecalis strain SJ82 chromosome, complete genome",UWD99825.1,DgaE family pyridoxal phosphate-dependent ammonia lyase,Vancomycin_rast_recs
